# STIVEN SALDAÑA EXAMEN


In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/preprocesamiento-lib/__script__.py
/kaggle/input/preprocesamiento-lib/__results__.html
/kaggle/input/preprocesamiento-lib/__script__.ipynb
/kaggle/input/preprocesamiento-lib/__output__.json
/kaggle/input/preprocesamiento-lib/custom.css
/kaggle/input/rotten-tomatoes-movies-and-critic-reviews-dataset/rotten_tomatoes_movies.csv
/kaggle/input/rotten-tomatoes-movies-and-critic-reviews-dataset/rotten_tomatoes_critic_reviews.csv


# INSTALACION DE DEPENDENCIAS

In [42]:
!pip install rank_bm25 --quiet

# CARGA DE DATOS

In [45]:
import pandas as pd
import numpy as np
import string
import nltk
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer
from rank_bm25 import BM25Okapi


# Preparacion

In [48]:
nltk.download('stopwords')
nltk.download('punkt')

def preprocesar_y_tokenizar(texto):
    if not isinstance(texto, str):
        return []
    
    texto = texto.lower()
    texto = texto.translate(str.maketrans('', '', string.punctuation))
    tokens = texto.split()
    
    stop_words = set(stopwords.words('english'))
    stemmer = PorterStemmer()
    
    return [stemmer.stem(word) for word in tokens if word not in stop_words]

[nltk_data] Downloading package stopwords to /usr/share/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /usr/share/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


# SE CARGAN DATOS
Aqui se cargan los datos y se eliminan comentarios o datos duplicados para evitar que salgan en el ranking de manera duplicad 

In [49]:
path_reviews = '/kaggle/input/rotten-tomatoes-movies-and-critic-reviews-dataset/rotten_tomatoes_critic_reviews.csv'
path_movies = '/kaggle/input/rotten-tomatoes-movies-and-critic-reviews-dataset/rotten_tomatoes_movies.csv'

try:
    df_reviews = pd.read_csv(path_reviews)
    df_movies = pd.read_csv(path_movies)
except:
    df_reviews = pd.DataFrame(columns=['rotten_tomatoes_link', 'review_content'])
    df_movies = pd.DataFrame(columns=['rotten_tomatoes_link', 'movie_title', 'movie_info', 'genres'])

df_full = pd.merge(df_reviews, df_movies, on='rotten_tomatoes_link', how='inner')
df_full['review_content'] = df_full['review_content'].fillna('')
df_full['movie_info'] = df_full['movie_info'].fillna('')
df_full['movie_title'] = df_full['movie_title'].fillna('')

print("Aqui se agrupa los datos")
df_grouped = df_full.groupby(['movie_title', 'movie_info', 'genres']).agg({
    'review_content': lambda x: " ".join(x.astype(str))
}).reset_index()

df_sample = df_grouped.head(1000).copy()

Aqui se agrupa los datos


# USO DEL MODELO BM25 

In [50]:
print("Se usa BM25")

df_sample['contenido_unificado'] = (
    df_sample['movie_title'] + " " + 
    df_sample['movie_title'] + " " + 
    df_sample['movie_info'] + " " + 
    df_sample['review_content']
)

corpus_tokenizado = df_sample['contenido_unificado'].apply(preprocesar_y_tokenizar).tolist()
bm25 = BM25Okapi(corpus_tokenizado, k1=3.0, b=0.1)

print("ok")

Se usa BM25
ok


In [51]:
def buscar_agresivo(consulta, top_n=5):
    query_tokens = preprocesar_y_tokenizar(consulta)
    scores = bm25.get_scores(query_tokens)
    
    top_indices = np.argsort(scores)[::-1][:top_n]
    
    print(f"\n Consulta: '{consulta}'")
    print("-" * 60)
    
    found = False
    for i in top_indices:
        score = scores[i]
        if score > 0: 
            found = True
            titulo = df_sample.iloc[i]['movie_title']
            genero = df_sample.iloc[i]['genres']
            print(f"[Score: {score:.2f}] {titulo}")
            print(f"               Género: {genero}")
            
    if not found:
        print("   (Sin resultados)")

In [52]:
consultas = ["Space travel", "Family animation", "Horror fear"]
for q in consultas:
    buscar_agresivo(q)


 Consulta: 'Space travel'
------------------------------------------------------------
[Score: 14.54] Ad Astra
               Género: Mystery & Suspense, Science Fiction & Fantasy
[Score: 9.93] A Sound of Thunder
               Género: Action & Adventure, Drama, Horror, Science Fiction & Fantasy
[Score: 9.88] Alien: Covenant
               Género: Action & Adventure, Mystery & Suspense, Science Fiction & Fantasy
[Score: 9.72] Aliens of the Deep
               Género: Documentary, Special Interest
[Score: 8.78] A Wrinkle in Time
               Género: Action & Adventure, Drama, Kids & Family, Science Fiction & Fantasy

 Consulta: 'Family animation'
------------------------------------------------------------
[Score: 10.95] Abominable
               Género: Action & Adventure, Animation, Comedy
[Score: 9.73] Aladdin
               Género: Action & Adventure, Kids & Family, Science Fiction & Fantasy
[Score: 8.95] African Cats
               Género: Action & Adventure, Documentary, Specia

1. Preprocesamiento de Datos
Se implementó la librería NLTK para la limpieza
Se agregaron todas las críticas en un único "macro-documento" para mejorar el sistema de recuperación según el contexto
Se optó por un Modelo Probabilístico por que

* BM25 evita que palabras repetidas sean más relevantes.
* Esto permite al sistema a ignorar la gran longitud de los documentos y priorizar la densidad de palabras clave.

3. Simulación de Consultas
El sistema respondió a las consultas de prueba ("Space travel", "Family movies").
* Los resultados mostraron puntuaciones altas mayor a 10.0
* El ranking priorizó correctamente películas donde el tema era central en la discusión de la audiencia, incluso si el título no contenía la palabra exacta.
La estrategia de duplicar el título en el índice fue efectiva para el del ranking

5. Posibles mejoras
Para mejoras Futuras se propone integrar Embeddings para capturar similitud de semántica en futuras versiones.
